In [6]:
!pip install tf-nightly-gpu

     |████████████████████████████████| 521.9MB 28kB/s 
     |████████████████████████████████| 3.0MB 40.7MB/s 
     |████████████████████████████████| 460kB 44.6MB/s 


In [0]:
# Imports
import numpy as np
import tensorflow as tf
import tensorflow.keras.optimizers
import tensorflow.keras.losses
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json
import os

In [2]:
print(tf.__version__)

2.2.0


## Import and check data

In [3]:
# Mount drive to access files

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Paths
train_y_path = "/content/drive/My Drive/DD2424Files/train_y.npz"
val_y_path   = "/content/drive/My Drive/DD2424Files/val_y.npz"
test_y_path  = "/content/drive/My Drive/DD2424Files/test_y.npz"

train_X_path = "/content/drive/My Drive/DD2424Files/train_X.npz"
val_X_path   = "/content/drive/My Drive/DD2424Files/val_X.npz"
test_X_path  = "/content/drive/My Drive/DD2424Files/test_X.npz"

train_X_mfcc_path = "/content/drive/My Drive/DD2424Files/train_X_mfcc.npz"
val_X_mfcc_path   = "/content/drive/My Drive/DD2424Files/val_X_mfcc.npz"
test_X_mfcc_path  = "/content/drive/My Drive/DD2424Files/test_X_mfcc.npz"

In [0]:
# Import training and validation data
train_y = np.load(train_y_path)['arr_0']
train_y = tf.one_hot(train_y, 8)

val_y = np.load(val_y_path)['arr_0']
val_y = tf.one_hot(val_y, 8)

train_X = tf.convert_to_tensor(np.expand_dims(np.load(train_X_path)['arr_0'], -1))
val_X = tf.convert_to_tensor(np.expand_dims(np.load(val_X_path)['arr_0'], -1))
#train_X_mfcc = np.load(train_X_mfcc_path)['arr_0']
#val_X_mfcc = np.load(val_X_mfcc_path)['arr_0']

In [6]:
# Check the data shape
print(train_X.shape)
print(val_X.shape)

(6397, 96, 1405, 1)
(800, 96, 1405, 1)


In [0]:
# Assert assumptions about data

MEL_BINS   = 96
TIME_STEPS = 1405

def check_data(X, y):
  assert len(y.shape) == 2
  assert len(X.shape) == 4
  assert y.shape[0] == X.shape[0]
  assert X.shape[1] == MEL_BINS
  assert X.shape[2] == TIME_STEPS
  assert X.shape[3] == 1

check_data(val_X, val_y)
check_data(train_X, train_y)

## The model

In [0]:
# All model definitions

def get_mlp_model():

  mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8)
  ])
  return mlp_model

#-------------------------------------------------------

def cnn1_model(hyperparams):
  NUMBER_FILTERS_1 = hyperparams.get("number_filters_1")
  KERNEL_SIZE = hyperparams.get("kernel_size_1")

  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(NUMBER_FILTERS_1, KERNEL_SIZE, padding = 'same', input_shape = (96, 1405, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8)                             
  ])
  return model


In [0]:
# Utility functions for saving outputs

def plot_history(history, val_string, loss_string):
  # Plot results
  plt.figure(1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.legend(['Accuracy', 'Validation accuracy'])
  plt.title('Accuracy')
  plt.savefig(val_string, dpi = 300)
  plt.close()

  plt.figure(2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(['Loss', 'Validation loss'])
  plt.title('Loss')
  plt.savefig(loss_string, dpi = 300)
  plt.close()

def filename():
	return str(datetime.datetime.now()).replace(':','.')[2:19]

In [0]:
# Run an experiment

def run_experiment(model, model_string, hyperparams):

  file_location = "/content/drive/My Drive/DD2424Files/Results/" + filename() + "/"
  os.mkdir(file_location)

  # Compile model

  optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparams.get("learning_rate"), epsilon=hyperparams.get("epsilon"))
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
  metrics = ['accuracy']

  model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

  # Train model
  BATCH_SIZE = hyperparams.get("batch_size")
  EPOCHS = hyperparams.get("epochs")
  tf.keras.backend.clear_session()

  save_callback = tf.keras.callbacks.ModelCheckpoint(
      file_location + "models_" + model_string, monitor='val_accuracy', verbose=0, save_best_only=True,
      save_weights_only=False, mode='max', save_freq='epoch')

  history = model.fit(train_X, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS,
                          validation_data=(val_X, val_y), verbose=1, callbacks = [save_callback])

  # Save stuff
  plot_history(history, file_location + "acc_" + model_string + ".png", file_location + "loss_" + model_string + ".png")
  json.dump(history.history, open(file_location + "history_" + model_string, 'w'))
  json.dump(hyperparams, open(file_location + "hyperparams_" + model_string, 'w'))

  return history




In [41]:
# Experiment 1

hyperparams = {"number_filters_1" : 15,
               "kernel_size_1" : (1, 4),
               "epsilon": 0.01, 
               "learning_rate": 0.01, 
               "batch_size": 32, 
               "epochs": 2}

model_string = "cnn15-fc8"
model = cnn1_model(hyperparams)

history = run_experiment(model, model_string, hyperparams)

15
(1, 4)
Epoch 1/2
200/200 [==============================] - 10s 51ms/step - loss: 19124.6992 - accuracy: 0.1458 - val_loss: 729.0200 - val_accuracy: 0.1737
Epoch 2/2
200/200 [==============================] - 11s 54ms/step - loss: 314.0475 - accuracy: 0.4491 - val_loss: 607.1850 - val_accuracy: 0.1825
